In [1]:
# %% import packages and load environment

import torch
import torch.nn as nn
import numpy as np
import gym
from torch.distributions import Normal
import scipy.signal
from torch.optim import SGD

env = gym.make('Pendulum-v1')

import os

folder_name = 'Data/Relu'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# %% define the policy NN and some functions


class GaussianPolicy(nn.Module):
    def __init__(self):
        super(GaussianPolicy, self).__init__()
        self.fc1 = nn.Linear(3, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x)) * 2

        return x

class GaussianPolicy(nn.Module):
    def __init__(self):
        super(GaussianPolicy, self).__init__()
        self.fc1 = nn.Linear(3, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x)) * 2

        return x

def get_policy(state, policy):
    # function that uses NN to calculate mean of normal distribution, returns the Normal objective

    state = torch.as_tensor(np.array(state), dtype=torch.float)
    mean = policy(state)
    return Normal(mean, 1)


def get_action(state, policy):
    # sample the Normal distribution and return the result as action

    action = get_policy(state, policy).sample()
    return action


def discount_cumsum(x: np.ndarray, gamma: float) -> torch.tensor:
    # copy from Ray v1.9.2 source code https://docs.ray.io/en/releases-1.9.2/_modules/ray/rllib/evaluation/postprocessing.html?highlight=discount_cumsum#
    """Calculates the discounted cumulative sum over a reward sequence `x`.

    y[t] - discount*y[t+1] = x[t]
    reversed(y)[t] - discount*reversed(y)[t-1] = reversed(x)[t]

    Args:
        gamma: The discount factor gamma.

    Returns:
        The sequence containing the discounted cumulative sums
        for each individual reward in `x` till the end of the trajectory.

    Examples:
        >>> x = np.array([0.0, 1.0, 2.0, 3.0])
        >>> gamma = 0.9
        >>> discount_cumsum(x, gamma)
        ... array([0.0 + 0.9*1.0 + 0.9^2*2.0 + 0.9^3*3.0,
        ...        1.0 + 0.9*2.0 + 0.9^2*3.0,
        ...        2.0 + 0.9*3.0,
        ...        3.0])
    """
    result = scipy.signal.lfilter([1], [1, float(-gamma)], x[::-1], axis=0)[::-1]
    return torch.as_tensor(result.copy())


def compute_objective(obs_traj, action_traj, reward_traj, policy, discount=0.99):
    H = len(reward_traj)
    Q_func = discount_cumsum(torch.Tensor.numpy(reward_traj), discount)
    weight = discount ** torch.arange(0, H)
    log_prob = get_policy(obs_traj, policy).log_prob(action_traj).reshape(-1)

    obj_func = Q_func * log_prob * weight
    return obj_func.sum()  # .mean()  #


def get_samples(policy, maximum_length=100):
    # get a single batch trajectory
    obs_traj = []
    action_traj = []
    reward_traj = []

    # reset the environment
    obs, _ = env.reset(seed=6)

    for _ in range(maximum_length):
        # take one step
        action = get_action(obs, policy)  # stochastic action
        # state = torch.as_tensor(np.array(obs), dtype=torch.float)
        # action = policy(state).detach() # determinastic action
        next_state, reward, done, _, _ = env.step(action)

        # save the result
        obs_traj.append(obs.copy())
        action_traj.append(action)
        reward_traj.append(reward)

        # update obs
        obs = next_state

    eps = [obs_traj, action_traj, reward_traj]
    return eps




def get_grad(model):
    # get gradient of a NN model
    temp_grad = []
    for param in model.parameters():
        temp_grad.append(param.grad.data.reshape(-1))
    return torch.cat([g for g in temp_grad])



In [ ]:

# %% save batch samples to local files
sample_batch = 5000

torch.manual_seed(6)
policy = GaussianPolicy()

for i in range(sample_batch):
    file_name = 'Data/Relu/Pendulum_data_maximum300_' + str(int(i)) + '.pt'
    eps = get_samples(policy, maximum_length=300)
    torch.save(eps, file_name)
    print('finish file No.%d\r' % i, end="")


In [ ]:
# %% save gradient
discount = 0.97
sample_batch = 5000

torch.manual_seed(6)
policy = GaussianPolicy()

# get length of parmeter
model_parameters = filter(lambda p: p.requires_grad, policy.parameters())
params_num = sum([np.prod(p.size()) for p in model_parameters])

eta = np.linspace(2, 300, 20, dtype=int)
sample_variance = np.zeros([len(eta), params_num])
sample_mean = np.zeros([len(eta), params_num])

optimizer = SGD(policy.parameters(), lr=0.00001)

# %% Calculate gradient for each sample
for i in range(len(eta)):
    H = eta[i]  # maximum length
    grad_sample = np.zeros([sample_batch, params_num])

    for j in range(sample_batch):
        file_name = 'Data/Relu/Pendulum_data_maximum300_' + str(int(j)) + '.pt'
        eps = torch.load(file_name)

        obs_traj = eps[0]
        action_traj = torch.as_tensor(eps[1]).reshape(-1, 1)
        reward_traj = torch.as_tensor(eps[2])

        optimizer.zero_grad()
        loss = - compute_objective(obs_traj[:H], action_traj[:H],
                                   reward_traj[:H], policy, discount)
        loss.backward()
        grad = get_grad(policy)
        grad_sample[j, :] = grad.numpy()
    file_name = 'Data/Relu/Pendulum_maximum300_gamma97_grad_H_' + \
        str(int(H)) + '.pt'
    torch.save(grad_sample, file_name)
    print('finish eta = %d\r' % H, end="")
